In [361]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import yfinance as yf

# Data Preprocessing

In [362]:
# Download historical stock data for Apple Inc. (AAPL)
ticker = 'AAPL' # Apple Inc.
start_date = '2020-01-01'
df = yf.download(ticker, start=start_date)

[*********************100%***********************]  1 of 1 completed


In [363]:
# Feature Engineering
df["log_return"] = np.log(df["Close"]).diff()
df["target"] = df["log_return"].shift(-1)

df["volatility_10"] = df["log_return"].rolling(10).std()
df["volatility_30"] = df["log_return"].rolling(30).std()

df["momentum_10"] = df["Close"].pct_change(10)
df["momentum_30"] = df["Close"].pct_change(30)

df["volume_z"] = (
    df["Volume"] - df["Volume"].rolling(30).mean()
) / df["Volume"].rolling(30).std()

df = df.dropna().reset_index(drop=True)


In [364]:
df

Price,Close,High,Low,Open,Volume,log_return,target,volatility_10,volatility_30,momentum_10,momentum_30,volume_z
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,,,,,,,
0,78.516335,78.765209,78.008919,78.465588,80113600,0.000246,-0.018480,0.014343,0.016556,0.052377,0.084472,-1.619249
1,77.078674,77.259894,76.017933,76.199152,152531200,-0.018480,0.014379,0.016060,0.016856,0.035952,0.075067,0.655929
2,78.194969,78.424517,77.320285,77.320285,93984000,0.014379,-0.010312,0.013200,0.016969,0.017369,0.082015,-1.134825
3,77.392792,78.443866,76.887794,77.955784,100566000,-0.010312,-0.022895,0.013491,0.017083,-0.001213,0.075975,-0.917206
4,75.640991,77.429029,75.024848,76.986849,129554000,-0.022895,-0.048666,0.014527,0.017491,-0.035107,0.034971,-0.028381
...,...,...,...,...,...,...,...,...,...,...,...,...
1501,275.652069,279.238709,272.974582,277.869995,52977400,-0.002100,0.007978,0.015807,0.013791,0.110972,0.018231,0.214674
1502,277.859985,280.647386,276.671095,276.860920,50453400,0.007978,-0.011729,0.015305,0.013833,0.121271,0.021148,0.050096
1503,274.619995,278.200012,271.700012,277.910004,44623400,-0.011729,-0.003429,0.015488,0.013986,0.076218,0.003897,-0.316881


In [365]:
# Define feature columns and target column
FEATURE_COLS = [
    "volatility_10",
    "volatility_30",
    "momentum_10",
    "momentum_30",
    "volume_z",
]

TARGET_COL = "target"
SEQ_LEN = 30


In [366]:
# Train-test split
split_idx = int(len(df) * 0.7)

train_df = df.iloc[:split_idx]
test_df = df.iloc[split_idx + SEQ_LEN:]


In [367]:
# Scale features and target
x_scaler = StandardScaler()
y_scaler = StandardScaler()

train_df[FEATURE_COLS] = x_scaler.fit_transform(train_df[FEATURE_COLS])
test_df[FEATURE_COLS] = x_scaler.transform(test_df[FEATURE_COLS])

train_df[TARGET_COL] = y_scaler.fit_transform(
    train_df[[TARGET_COL]]
)
test_df[TARGET_COL] = y_scaler.transform(
    test_df[[TARGET_COL]]
)


C:\Users\HP\AppData\Local\Temp\ipykernel_4656\1918266380.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[FEATURE_COLS] = x_scaler.fit_transform(train_df[FEATURE_COLS])
C:\Users\HP\AppData\Local\Temp\ipykernel_4656\1918266380.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[FEATURE_COLS] = x_scaler.transform(test_df[FEATURE_COLS])
C:\Users\HP\AppData\Local\Temp\ipykernel_4656\1918266380.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

In [368]:
# Define Dataset
class SequenceDataset(Dataset):
    def __init__(self, df, feature_cols, target_col, seq_len):
        self.X = df[feature_cols].values
        self.y = df[target_col].values
        self.seq_len = seq_len

    def __len__(self):
        return len(self.X) - self.seq_len

    def __getitem__(self, idx):
        x = self.X[idx : idx + self.seq_len]
        y = self.y[idx + self.seq_len]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)


In [369]:
# Create DataLoaders
train_ds = SequenceDataset(train_df, FEATURE_COLS, TARGET_COL, SEQ_LEN)
test_ds = SequenceDataset(test_df, FEATURE_COLS, TARGET_COL, SEQ_LEN)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=64, shuffle=False)


# LSTM

In [370]:
# Define LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, input_size):
        super().__init__()

        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=32,
            num_layers=1,
            batch_first=True
        )

        self.fc = nn.Linear(32, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        return self.fc(out).squeeze()


In [371]:
# Train the model
device = "cuda" if torch.cuda.is_available() else "cpu"

model = LSTMModel(input_size=len(FEATURE_COLS)).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=1e-3,
    weight_decay=1e-4
)


In [372]:
# Training loop
EPOCHS = 25

for epoch in range(EPOCHS):
    model.train()
    train_losses = []

    for x, y in train_loader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        preds = model(x)
        loss = criterion(preds, y)
        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())

    print(f"Epoch {epoch+1}: Train MSE = {np.mean(train_losses):.4f}")


Epoch 1: Train MSE = 0.8117
Epoch 2: Train MSE = 0.8047
Epoch 3: Train MSE = 0.8036
Epoch 4: Train MSE = 0.8033
Epoch 5: Train MSE = 0.8028
Epoch 6: Train MSE = 0.8022
Epoch 7: Train MSE = 0.8016
Epoch 8: Train MSE = 0.8010
Epoch 9: Train MSE = 0.8002
Epoch 10: Train MSE = 0.7994
Epoch 11: Train MSE = 0.7985
Epoch 12: Train MSE = 0.7974
Epoch 13: Train MSE = 0.7962
Epoch 14: Train MSE = 0.7949
Epoch 15: Train MSE = 0.7933
Epoch 16: Train MSE = 0.7926
Epoch 17: Train MSE = 0.7947
Epoch 18: Train MSE = 0.7913
Epoch 19: Train MSE = 0.7892
Epoch 20: Train MSE = 0.7880
Epoch 21: Train MSE = 0.7902
Epoch 22: Train MSE = 0.7883
Epoch 23: Train MSE = 0.7880
Epoch 24: Train MSE = 0.7839
Epoch 25: Train MSE = 0.7829


In [373]:
# Evaluate the model
model.eval()

preds, trues = [], []

with torch.no_grad():
    for x, y in test_loader:
        x = x.to(device)
        pred = model(x).cpu().numpy()
        preds.append(pred)
        trues.append(y.numpy())

preds = np.concatenate(preds)
trues = np.concatenate(trues)

# Inverse scale
preds_real = y_scaler.inverse_transform(preds.reshape(-1,1)).flatten()
trues_real = y_scaler.inverse_transform(trues.reshape(-1,1)).flatten()


In [374]:
# Test MSE
test_mse = np.mean((trues-preds)**2)
print(f"Test MSE = {test_mse:.4f}")

Test MSE = 0.7595


In [375]:
# Volatility target for scaling (1% daily is conservative)
vol_target = 0.01

# Smooth bounded position sizing
position = np.tanh(preds_real / vol_target)

print("Position stats:")
print("  min:", position.min())
print("  max:", position.max())
print("  std:", position.std())


Position stats:
  min: -0.53992736
  max: 0.6287636
  std: 0.14107177


In [376]:
# Cost per unit turnover (5 bps = realistic for equities)
cost_per_trade = 0.0005

# Turnover = absolute position change
turnover = np.abs(np.diff(position, prepend=0))

# Transaction costs
costs = cost_per_trade * turnover

print("Average daily turnover:", turnover.mean())

Average daily turnover: 0.04223890200651391


In [377]:
# Calculate returns
gross_returns = position * trues_real
net_returns = gross_returns - costs

In [378]:
# Performance metrics
def sharpe_ratio(returns, annualize=True):
    if returns.std() == 0:
        return 0.0
    sr = returns.mean() / returns.std()
    return sr * np.sqrt(252) if annualize else sr


sharpe_gross = sharpe_ratio(gross_returns)
sharpe_net = sharpe_ratio(net_returns)

direction_accuracy = np.mean(
    np.sign(position) == np.sign(trues_real)
)

rmse = np.sqrt(np.mean((preds_real - trues_real) ** 2))
corr = np.corrcoef(preds_real, trues_real)[0, 1]

print("\n=== PERFORMANCE ===")
print(f"RMSE: {rmse:.6f}")
print(f"Correlation: {corr:.4f}")
print(f"Direction Accuracy: {direction_accuracy:.4f}")
print(f"Sharpe (Gross): {sharpe_gross:.3f}")
print(f"Sharpe (Net): {sharpe_net:.3f}")


=== PERFORMANCE ===
RMSE: 0.018112
Correlation: 0.0444
Direction Accuracy: 0.4898
Sharpe (Gross): 0.366
Sharpe (Net): 0.305


# LSTM+Attention

In [379]:
# Define Temporal Attention Mechanism
class TemporalAttention(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.score = nn.Linear(hidden_size, 1, bias=False)

    def forward(self, lstm_out):
        # lstm_out: (batch, seq_len, hidden)
        scores = self.score(lstm_out).squeeze(-1)  # (batch, seq_len)
        weights = torch.softmax(scores, dim=1)     # stable
        context = torch.sum(lstm_out * weights.unsqueeze(-1), dim=1)
        return context

In [380]:
# Define LSTM with Attention Model
class AttnLSTM(nn.Module):
    def __init__(self, input_size):
        super().__init__()

        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=32,
            num_layers=1,
            batch_first=True
        )

        self.attn = TemporalAttention(hidden_size=32)
        self.fc = nn.Linear(32, 1)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        context = self.attn(lstm_out)
        return self.fc(context).squeeze()


In [381]:
# Train the model with Attention
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=1e-3,
    weight_decay=1e-4
)

for epoch in range(EPOCHS):
    model.train()
    losses = []

    for x, y in train_loader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        preds = model(x)
        loss = criterion(preds, y)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        losses.append(loss.item())

    print(f"Epoch {epoch+1}: Train MSE = {np.mean(losses):.4f}")


Epoch 1: Train MSE = 0.7833
Epoch 2: Train MSE = 0.7819
Epoch 3: Train MSE = 0.7781
Epoch 4: Train MSE = 0.7765
Epoch 5: Train MSE = 0.7758
Epoch 6: Train MSE = 0.7746
Epoch 7: Train MSE = 0.7744
Epoch 8: Train MSE = 0.7720
Epoch 9: Train MSE = 0.7698
Epoch 10: Train MSE = 0.7706
Epoch 11: Train MSE = 0.7663
Epoch 12: Train MSE = 0.7649
Epoch 13: Train MSE = 0.7659
Epoch 14: Train MSE = 0.7616
Epoch 15: Train MSE = 0.7649
Epoch 16: Train MSE = 0.7577
Epoch 17: Train MSE = 0.7560
Epoch 18: Train MSE = 0.7569
Epoch 19: Train MSE = 0.7518
Epoch 20: Train MSE = 0.7513
Epoch 21: Train MSE = 0.7479
Epoch 22: Train MSE = 0.7471
Epoch 23: Train MSE = 0.7437
Epoch 24: Train MSE = 0.7415
Epoch 25: Train MSE = 0.7383


In [382]:
# Evaluate the model
model.eval()

preds, trues = [], []

with torch.no_grad():
    for x, y in test_loader:
        x = x.to(device)
        pred = model(x).cpu().numpy()
        preds.append(pred)
        trues.append(y.numpy())

preds = np.concatenate(preds)
trues = np.concatenate(trues)

# Inverse scale
preds_real = y_scaler.inverse_transform(preds.reshape(-1,1)).flatten()
trues_real = y_scaler.inverse_transform(trues.reshape(-1,1)).flatten()

In [383]:
# Test MSE
test_mse = np.mean((trues-preds)**2)
print(f"Test MSE = {test_mse:.4f}")

Test MSE = 0.7474


In [384]:
# Volatility target for scaling (1% daily is conservative)
vol_target = 0.01

# Smooth bounded position sizing
position = np.tanh(preds_real / vol_target)

print("Position stats:")
print("  min:", position.min())
print("  max:", position.max())
print("  std:", position.std())

Position stats:
  min: -0.8888539
  max: 0.96449184
  std: 0.31432658


In [385]:
# Cost per unit turnover (5 bps = realistic for equities)
cost_per_trade = 0.0005

# Turnover = absolute position change
turnover = np.abs(np.diff(position, prepend=0))

# Transaction costs
costs = cost_per_trade * turnover

print("Average daily turnover:", turnover.mean())

Average daily turnover: 0.06904630720133095


In [386]:
# Calculate returns
gross_returns = position * trues_real
net_returns = gross_returns - costs

In [387]:
# Performance metrics
def sharpe_ratio(returns, annualize=True):
    if returns.std() == 0:
        return 0.0
    sr = returns.mean() / returns.std()
    return sr * np.sqrt(252) if annualize else sr


sharpe_gross = sharpe_ratio(gross_returns)
sharpe_net = sharpe_ratio(net_returns)

direction_accuracy = np.mean(
    np.sign(position) == np.sign(trues_real)
)

rmse = np.sqrt(np.mean((preds_real - trues_real) ** 2))
corr = np.corrcoef(preds_real, trues_real)[0, 1]

print("\n=== PERFORMANCE ===")
print(f"RMSE: {rmse:.6f}")
print(f"Correlation: {corr:.4f}")
print(f"Direction Accuracy: {direction_accuracy:.4f}")
print(f"Sharpe (Gross): {sharpe_gross:.3f}")
print(f"Sharpe (Net): {sharpe_net:.3f}")



=== PERFORMANCE ===
RMSE: 0.017966
Correlation: 0.1476
Direction Accuracy: 0.5102
Sharpe (Gross): 1.102
Sharpe (Net): 1.042


Based on RMSE, the LSTM+Attention is better. Based on Sharpe, the LSTM+Attention produces cleaner or more tradable signal. So, for the production we choose LSTM+Attention. 

## Walk-Forward Validation

In [388]:
def add_regime_features(df):
    df = df.copy()

    df["ret"] = df["Close"].pct_change()
    df["vol_20"] = df["ret"].rolling(20).std()
    df["vol_60_med"] = df["vol_20"].rolling(60).median()
    df["vol_ok"] = df["vol_20"] < 1.2 * df["vol_60_med"]

    close = df["Close"].iloc[:, 0]
    df["ma50"] = close.rolling(50).mean()
    df["trend_strength"] = (close - df["ma50"]).abs() / df["ma50"]
    df["trend_ok"] = df["trend_strength"] > 0.0075

    df["regime_ok"] = df["vol_ok"] & df["trend_ok"]

    return df


In [389]:
def train_and_predict(train_df, test_df, features, target, seq_len, epochs=15):
    x_scaler = StandardScaler()
    y_scaler = StandardScaler()

    train_df = train_df.copy()
    test_df = test_df.copy()

    train_df[features] = x_scaler.fit_transform(train_df[features])
    test_df[features] = x_scaler.transform(test_df[features])

    train_df[target] = y_scaler.fit_transform(train_df[[target]])
    test_df[target] = y_scaler.transform(test_df[[target]])

    train_ds = SequenceDataset(train_df, features, target, seq_len)
    test_ds = SequenceDataset(test_df, features, target, seq_len)

    train_loader = DataLoader(train_ds, batch_size=64, shuffle=False)
    test_loader = DataLoader(test_ds, batch_size=64, shuffle=False)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = AttnLSTM(len(features)).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
    loss_fn = nn.MSELoss()

    for _ in range(epochs):
        model.train()
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            loss = loss_fn(model(x), y)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

    model.eval()
    preds, trues = [], []

    with torch.no_grad():
        for x, y in test_loader:
            preds.append(model(x.to(device)).cpu().numpy())
            trues.append(y.numpy())

    preds = np.concatenate(preds)
    trues = np.concatenate(trues)

    preds = y_scaler.inverse_transform(preds.reshape(-1, 1)).flatten()
    trues = y_scaler.inverse_transform(trues.reshape(-1, 1)).flatten()

    return preds, trues


In [390]:
def walk_forward(df, features, target):
    SEQ_LEN = 30
    TRAIN = 756
    TEST = 63

    vol_target = 0.01
    cost = 0.0005
    eps = 1e-8

    all_returns = []
    start = 0

    while start + TRAIN + TEST < len(df):
        train_df = df.iloc[start:start + TRAIN].copy()
        test_df  = df.iloc[start + TRAIN:start + TRAIN + TEST].copy()

        preds, trues = train_and_predict(
            train_df, test_df, features, target, SEQ_LEN
        )

        preds = np.asarray(preds, dtype=float)
        trues = np.asarray(trues, dtype=float)

        # 1️⃣ Prediction smoothing (SAFE)
        preds = pd.Series(preds).rolling(3, min_periods=1).mean().values

        # 2️⃣ Raw signal
        raw_pos = np.tanh(preds / vol_target)

        # 3️⃣ Confidence gating
        thresh = np.nanpercentile(np.abs(preds), 60)
        gate = np.abs(preds) > thresh
        raw_pos *= gate.astype(float)

        # 4️⃣ Volatility targeting (SAFE)
        realized_vol = test_df["vol_20"].iloc[-len(raw_pos):].values
        realized_vol = np.nan_to_num(realized_vol, nan=vol_target, posinf=vol_target)
        realized_vol = np.maximum(realized_vol, eps)

        pos = raw_pos * (vol_target / realized_vol)
        pos = np.clip(pos, -1, 1)

        # 5️⃣ Regime filter (SAFE)
        regime = test_df["regime_ok"].iloc[-len(pos):].values
        regime = np.nan_to_num(regime, nan=0.0)
        pos *= regime.astype(float)

        # 6️⃣ Costs
        turnover = np.abs(np.diff(pos, prepend=0))
        costs = cost * turnover

        net_ret = pos * trues - costs
        net_ret = np.nan_to_num(net_ret)

        all_returns.append(net_ret)

        start += TEST

    return np.concatenate(all_returns)


In [391]:
def sharpe(r):
    return r.mean() / r.std() * np.sqrt(252)

def max_drawdown(r):
    cum = np.cumsum(r)
    return (cum - np.maximum.accumulate(cum)).min()


In [392]:
TRADE_START = 100
FEATURES = FEATURE_COLS

df_wf = add_regime_features(df)
df_wf = df_wf[TRADE_START:]
df_wf = df_wf.dropna().reset_index(drop=True)

returns = walk_forward(df_wf, FEATURES, "target")

print("WF Sharpe:", sharpe(returns))
print("Max Drawdown:", max_drawdown(returns))

WF Sharpe: 0.34100132795015486
Max Drawdown: -0.015244392432147823


## Production Check

In [393]:
# Production Drawdown
cum_returns = np.cumsum(net_returns)
running_max = np.maximum.accumulate(cum_returns)
drawdown = cum_returns - running_max

print("Max Drawdown:", drawdown.min())


Max Drawdown: -0.060040805297102616


At the worst point, the strategy was down ~10.2% from its previous peak.

In [394]:
print("\n=== COST SENSITIVITY CHECK===")
for cost in [0.0003, 0.0005, 0.001]:
    net = gross_returns - cost * turnover
    print(f"Cost {cost:.4f} → Sharpe {sharpe_ratio(net):.3f}")



=== COST SENSITIVITY CHECK===
Cost 0.0003 → Sharpe 1.066
Cost 0.0005 → Sharpe 1.042
Cost 0.0010 → Sharpe 0.982


Stable.

In [395]:
mid = len(net_returns) // 2

sr_first = sharpe_ratio(net_returns[:mid])
sr_second = sharpe_ratio(net_returns[mid:])

print("\n=== SUBPERIOD STABILITY SHARPE ===")
print("First half:", sr_first)
print("Second half:", sr_second)



=== SUBPERIOD STABILITY SHARPE ===
First half: 0.8969596013608857
Second half: 1.7547218750295126


All positives, supporting production confidence.